In [ ]:
%load_ext autoreload
%autoreload 2

# wSAA

> Module description for wSAA classes

In [ ]:
#| default_exp wSAA

In [ ]:
#| hide
from nbdev.showdoc import *

# from nbdev.qmd import *

## Packages

In [ ]:
#| export
from __future__ import annotations
from fastcore.docments import *
from fastcore.test import *
from fastcore.utils import *

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

from dddex.core import BaseWeightsBasedPredictor, restructureWeightsDataList

## wSAA - Random Forest

In [ ]:
#|export 

class RandomForestWSAA(RandomForestRegressor, BaseWeightsBasedPredictor):
        
    def fit(self, X, Y):

        super(RandomForestRegressor, self).fit(X = X, y = Y)
        
        self.Y = Y
        self.leafIndicesTrain = self.apply(X)
    
    #---
    
    def pointPredict(self, X):
        
        preds = super(RandomForestRegressor, self).predict(X)
        
        return preds
        

In [ ]:
show_doc(RandomForestWSAA)

/home/kagu/.conda/envs/HC-Scheduling/lib/python3.8/site-packages/fastcore/docscrape.py:225: UserWarning: Unknown section Attributes
  else: warn(msg)
/home/kagu/.conda/envs/HC-Scheduling/lib/python3.8/site-packages/fastcore/docscrape.py:225: UserWarning: Unknown section See Also
  else: warn(msg)
/home/kagu/.conda/envs/HC-Scheduling/lib/python3.8/site-packages/fastcore/docscrape.py:225: UserWarning: Unknown section Notes
  else: warn(msg)
/home/kagu/.conda/envs/HC-Scheduling/lib/python3.8/site-packages/fastcore/docscrape.py:225: UserWarning: Unknown section References
  else: warn(msg)
/home/kagu/.conda/envs/HC-Scheduling/lib/python3.8/site-packages/fastcore/docscrape.py:225: UserWarning: Unknown section Examples
  else: warn(msg)


---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/wSAA.py#L19){target="_blank" style="float:right; font-size:smaller"}

### RandomForestWSAA

>      RandomForestWSAA (n_estimators=100, criterion='mse', max_depth=None,
>                        min_samples_split=2, min_samples_leaf=1,
>                        min_weight_fraction_leaf=0.0, max_features='auto',
>                        max_leaf_nodes=None, min_impurity_decrease=0.0,
>                        min_impurity_split=None, bootstrap=True,
>                        oob_score=False, n_jobs=None, random_state=None,
>                        verbose=0, warm_start=False, ccp_alpha=0.0,
>                        max_samples=None)

A random forest regressor.

A random forest is a meta estimator that fits a number of classifying
decision trees on various sub-samples of the dataset and uses averaging
to improve the predictive accuracy and control over-fitting.
The sub-sample size is controlled with the `max_samples` parameter if
`bootstrap=True` (default), otherwise the whole dataset is used to build
each tree.

Read more in the :ref:`User Guide <forest>`.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| n_estimators | int | 100 | The number of trees in the forest.<br><br>.. versionchanged:: 0.22<br>   The default value of ``n_estimators`` changed from 10 to 100<br>   in 0.22. |
| criterion | str | mse | The function to measure the quality of a split. Supported criteria<br>are "mse" for the mean squared error, which is equal to variance<br>reduction as feature selection criterion, and "mae" for the mean<br>absolute error.<br><br>.. versionadded:: 0.18<br>   Mean Absolute Error (MAE) criterion. |
| max_depth | NoneType | None | The maximum depth of the tree. If None, then nodes are expanded until<br>all leaves are pure or until all leaves contain less than<br>min_samples_split samples. |
| min_samples_split | int | 2 | The minimum number of samples required to split an internal node:<br><br>- If int, then consider `min_samples_split` as the minimum number.<br>- If float, then `min_samples_split` is a fraction and<br>  `ceil(min_samples_split * n_samples)` are the minimum<br>  number of samples for each split.<br><br>.. versionchanged:: 0.18<br>   Added float values for fractions. |
| min_samples_leaf | int | 1 | The minimum number of samples required to be at a leaf node.<br>A split point at any depth will only be considered if it leaves at<br>least ``min_samples_leaf`` training samples in each of the left and<br>right branches.  This may have the effect of smoothing the model,<br>especially in regression.<br><br>- If int, then consider `min_samples_leaf` as the minimum number.<br>- If float, then `min_samples_leaf` is a fraction and<br>  `ceil(min_samples_leaf * n_samples)` are the minimum<br>  number of samples for each node.<br><br>.. versionchanged:: 0.18<br>   Added float values for fractions. |
| min_weight_fraction_leaf | float | 0.0 | The minimum weighted fraction of the sum total of weights (of all<br>the input samples) required to be at a leaf node. Samples have<br>equal weight when sample_weight is not provided. |
| max_features | str | auto | The number of features to consider when looking for the best split:<br><br>- If int, then consider `max_features` features at each split.<br>- If float, then `max_features` is a fraction and<br>  `int(max_features * n_features)` features are considered at each<br>  split.<br>- If "auto", then `max_features=n_features`.<br>- If "sqrt", then `max_features=sqrt(n_features)`.<br>- If "log2", then `max_features=log2(n_features)`.<br>- If None, then `max_features=n_features`.<br><br>Note: the search for a split does not stop until at least one<br>valid partition of the node samples is found, even if it requires to<br>effectively inspect more than ``max_features`` features. |
| max_leaf_nodes | NoneType | None | Grow trees with ``max_leaf_nodes`` in best-first fashion.<br>Best nodes are defined as relative reduction in impurity.<br>If None then unlimited number of leaf nodes. |
| min_impurity_decrease | float | 0.0 | A node will be split if this split induces a decrease of the impurity<br>greater than or equal to this value.<br><br>The weighted impurity decrease equation is the following::<br><br>    N_t / N * (impurity - N_t_R / N_t * right_impurity<br>                        - N_t_L / N_t * left_impurity)<br><br>where ``N`` is the total number of samples, ``N_t`` is the number of<br>samples at the current node, ``N_t_L`` is the number of samples in the<br>left child, and ``N_t_R`` is the number of samples in the right child.<br><br>``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,<br>if ``sample_weight`` is passed.<br><br>.. versionadded:: 0.19 |
| min_impurity_split | NoneType | None | Threshold for early stopping in tree growth. A node will split<br>if its impurity is above the threshold, otherwise it is a leaf.<br><br>.. deprecated:: 0.19<br>   ``min_impurity_split`` has been deprecated in favor of<br>   ``min_impurity_decrease`` in 0.19. The default value of<br>   ``min_impurity_split`` has changed from 1e-7 to 0 in 0.23 and it<br>   will be removed in 0.25. Use ``min_impurity_decrease`` instead. |
| bootstrap | bool | True | Whether bootstrap samples are used when building trees. If False, the<br>whole dataset is used to build each tree. |
| oob_score | bool | False | whether to use out-of-bag samples to estimate<br>the R\^2 on unseen data. |
| n_jobs | NoneType | None | The number of jobs to run in parallel. :meth:`fit`, :meth:`predict`,<br>:meth:`decision_path` and :meth:`apply` are all parallelized over the<br>trees. ``None`` means 1 unless in a :obj:`joblib.parallel_backend`<br>context. ``-1`` means using all processors. See :term:`Glossary<br><n_jobs>` for more details. |
| random_state | NoneType | None | Controls both the randomness of the bootstrapping of the samples used<br>when building trees (if ``bootstrap=True``) and the sampling of the<br>features to consider when looking for the best split at each node<br>(if ``max_features < n_features``).<br>See :term:`Glossary <random_state>` for details. |
| verbose | int | 0 | Controls the verbosity when fitting and predicting. |
| warm_start | bool | False | When set to ``True``, reuse the solution of the previous call to fit<br>and add more estimators to the ensemble, otherwise, just fit a whole<br>new forest. See :term:`the Glossary <warm_start>`. |
| ccp_alpha | float | 0.0 | Complexity parameter used for Minimal Cost-Complexity Pruning. The<br>subtree with the largest cost complexity that is smaller than<br>``ccp_alpha`` will be chosen. By default, no pruning is performed. See<br>:ref:`minimal_cost_complexity_pruning` for details.<br><br>.. versionadded:: 0.22 |
| max_samples | NoneType | None | If bootstrap is True, the number of samples to draw from X<br>to train each base estimator.<br><br>- If None (default), then draw `X.shape[0]` samples.<br>- If int, then draw `max_samples` samples.<br>- If float, then draw `max_samples * X.shape[0]` samples. Thus,<br>  `max_samples` should be in the interval `(0, 1)`.<br><br>.. versionadded:: 0.22 |

In [ ]:
show_doc(RandomForestWSAA.fit)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/wSAA.py#L21){target="_blank" style="float:right; font-size:smaller"}

### RandomForestWSAA.fit

>      RandomForestWSAA.fit (X, Y)

Build a forest of trees from the training set (X, y).

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| X | {array-like, sparse matrix} of shape (n_samples, n_features) | The training input samples. Internally, its dtype will be converted<br>to ``dtype=np.float32``. If a sparse matrix is provided, it will be<br>converted into a sparse ``csc_matrix``. |
| Y |  |  |
| **Returns** | **object** |  |

In [ ]:
#|export

@patch
def getWeightsData(self: RandomForestWSAA, 
                   X: np.ndarray, # Feature matrix for whose rows conditional density estimates are computed.
                   outputType: 'all' | # Specifies structure of output.
                               'onlyPositiveWeights' | 
                               'summarized' | 
                               'cumulativeDistribution' | 
                               'cumulativeDistributionSummarized' = 'onlyPositiveWeights', 
                   scalingList: list | np.ndarray | None = None, # List or array with same size as self.Y containing floats being multiplied with self.Y.
                   ):

    leafIndicesDf = self.apply(X)

    weightsDataList = list()

    for leafIndices in leafIndicesDf:
        leafComparisonMatrix = (self.leafIndicesTrain == leafIndices) * 1
        nObsInSameLeaf = np.sum(leafComparisonMatrix, axis = 0)

        # It can happen that RF decides that the best strategy is to fit no tree at
        # all and simply average all results (happens when min_child_sample is too high, for example).
        # In this case 'leafComparisonMatrix' mustn't be averaged because there has been only a single tree.
        if len(leafComparisonMatrix.shape) == 1:
            weights = leafComparisonMatrix / nObsInSameLeaf
        else:
            weights = np.mean(leafComparisonMatrix / nObsInSameLeaf, axis = 1)

        weightsPosIndex = np.where(weights > 0)[0]

        weightsDataList.append((weights[weightsPosIndex], weightsPosIndex))

    #---

    weightsDataList = restructureWeightsDataList(weightsDataList = weightsDataList, 
                                                 outputType = outputType, 
                                                 Y = self.Y, 
                                                 scalingList = scalingList,
                                                 equalWeights = False)

    return weightsDataList

In [ ]:
show_doc(RandomForestWSAA.getWeightsData)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/wSAA.py#L39){target="_blank" style="float:right; font-size:smaller"}

### RandomForestWSAA.getWeightsData

>      RandomForestWSAA.getWeightsData (X:numpy.ndarray, outputType:Union[Forwar
>                                       dRef('all'),ForwardRef('onlyPositiveWeig
>                                       hts'),ForwardRef('summarized'),ForwardRe
>                                       f('cumulativeDistribution'),ForwardRef('
>                                       cumulativeDistributionSummarized')]='onl
>                                       yPositiveWeights', scalingList:Union[lis
>                                       t,numpy.ndarray,NoneType]=None)

Compute weights of feature array X

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| X | np.ndarray |  | Feature matrix for whose rows conditional density estimates are computed. |
| outputType | 'all' \| 'onlyPositiveWeights' \| 'summarized' \| 'cumulativeDistribution' \| 'cumulativeDistributionSummarized' | onlyPositiveWeights | Specifies structure of output. |
| scalingList | list \| np.ndarray \| None | None | List or array with same size as self.Y containing floats being multiplied with self.Y. |

In [ ]:
#| export

@patch
def predict(self: RandomForestWSAA, 
            X: np.ndarray, # Feature matrix of samples for which an estimation of conditional quantiles is computed.
            probs: list | np.ndarray = [0.1, 0.5, 0.9], # Probabilities for which the estimated conditional p-quantiles are computed.
            outputAsDf: bool = False, # Output is either a dataframe with 'probs' as cols or a dict with 'probs' as keys.
            scalingList: list | np.ndarray | None = None, # List or array with same size as self.Y containing floats being multiplied with self.Y.
            ):

    quantileRes = super(BaseWeightsBasedPredictor, self).predict(X = X,
                                                                 probs = probs,
                                                                 outputAsDf = outputAsDf,
                                                                 scalingList = scalingList)

    return quantileRes

In [ ]:
show_doc(RandomForestWSAA.predict)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/wSAA.py#L81){target="_blank" style="float:right; font-size:smaller"}

### RandomForestWSAA.predict

>      RandomForestWSAA.predict (X:numpy.ndarray,
>                                probs:Union[list,numpy.ndarray]=[0.1, 0.5,
>                                0.9], outputAsDf:bool=False, scalingList:Union[
>                                list,numpy.ndarray,NoneType]=None)

Predict regression target for X.

The predicted regression target of an input sample is computed as the
mean predicted regression targets of the trees in the forest.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| X | np.ndarray |  | Feature matrix of samples for which an estimation of conditional quantiles is computed. |
| probs | list \| np.ndarray | [0.1, 0.5, 0.9] | Probabilities for which the estimated conditional p-quantiles are computed. |
| outputAsDf | bool | False | Output is either a dataframe with 'probs' as cols or a dict with 'probs' as keys. |
| scalingList | list \| np.ndarray \| None | None | List or array with same size as self.Y containing floats being multiplied with self.Y. |
| **Returns** | **ndarray of shape (n_samples,) or (n_samples, n_outputs)** |  | **The predicted values.** |

## SAA

In [ ]:
#| export

class SAA(BaseWeightsBasedPredictor):
    
    def __init__(self):
        
        self.Y = None
    

In [ ]:
show_doc(SAA)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/wSAA.py#L96){target="_blank" style="float:right; font-size:smaller"}

### SAA

>      SAA ()

Helper class that provides a standard way to create an ABC using
inheritance.

In [ ]:
#| export

@patch
def fit(self: SAA, 
        Y: np.ndarray, # Target values which form the estimated density function based on the SAA algorithm.
        ):
    self.Y = Y

In [ ]:
show_doc(SAA.fit)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/wSAA.py#L105){target="_blank" style="float:right; font-size:smaller"}

### SAA.fit

>      SAA.fit (Y:numpy.ndarray)

Fit weights-based predictor on given training data

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| Y | np.ndarray | Target values which form the estimated density function based on the SAA algorithm. |

In [ ]:
#| export


@patch
def getWeightsData(self: SAA, 
                   X: np.ndarray, # Feature matrix for whose rows conditional density estimates are computed.
                   outputType: 'all' | # Specifies structure of output.
                               'onlyPositiveWeights' | 
                               'summarized' | 
                               'cumulativeDistribution' | 
                               'cumulativeDistributionSummarized' = 'onlyPositiveWeights', 
                   scalingList: list | np.ndarray | None = None, # List or array with same size as self.Y containing floats being multiplied with self.Y.
                   ):

    if X is None:
        neighborsList = [np.arange(len(self.Y))]
    else:
        neighborsList = [np.arange(len(self.Y)) for i in range(X.shape[0])]

    # weightsDataList is a list whose elements correspond to one test prediction each. 
    weightsDataList = [(np.repeat(1 / len(neighbors), len(neighbors)), np.array(neighbors)) for neighbors in neighborsList]

    weightsDataList = restructureWeightsDataList(weightsDataList = weightsDataList, 
                                                 outputType = outputType, 
                                                 Y = self.Y,
                                                 scalingList = scalingList,
                                                 equalWeights = True)

    return weightsDataList

In [ ]:
show_doc(SAA.getWeightsData)

---

[source](https://github.com/kaiguender/dddex/blob/main/dddex/wSAA.py#L112){target="_blank" style="float:right; font-size:smaller"}

### SAA.getWeightsData

>      SAA.getWeightsData (X:numpy.ndarray, outputType:Union[ForwardRef('all'),F
>                          orwardRef('onlyPositiveWeights'),ForwardRef('summariz
>                          ed'),ForwardRef('cumulativeDistribution'),ForwardRef(
>                          'cumulativeDistributionSummarized')]='onlyPositiveWei
>                          ghts',
>                          scalingList:Union[list,numpy.ndarray,NoneType]=None)

Compute weights of feature array X

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| X | np.ndarray |  | Feature matrix for whose rows conditional density estimates are computed. |
| outputType | 'all' \| 'onlyPositiveWeights' \| 'summarized' \| 'cumulativeDistribution' \| 'cumulativeDistributionSummarized' | onlyPositiveWeights | Specifies structure of output. |
| scalingList | list \| np.ndarray \| None | None | List or array with same size as self.Y containing floats being multiplied with self.Y. |

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()